In [1]:
import os
import json
import re
import numpy as np
import pandas as pd
import sqlite3 as sql
import seaborn as sns
import difflib

### 1. Product

In [2]:
products = pd.read_csv("Cung cap HV/ProductRaw.csv")
products.head(2)

,item_id,name,description,rating,price,list_price,brand,group,url,image
0,48102821,Tai nghe Bluetooth Inpods 12 - Cảm biến vân ta...,THÔNG TIN CHI TIẾT\nDung lượng pin 300\nThời g...,4.0,77000,300000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-inpods-12-cam-bien-...,https://salt.tikicdn.com/cache/280x280/ts/prod...
1,52333193,Tai nghe bluetooth không dây F9 True wireless ...,THÔNG TIN CHI TIẾT\nDung lượng pin 2000mah\nTh...,4.5,132000,750000,OEM,Thiết Bị Số - Phụ Kiện Số/Thiết Bị Âm Thanh và...,https://tai-nghe-bluetooth-khong-day-f9-true-w...,https://salt.tikicdn.com/cache/280x280/ts/prod...


In [3]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4404 entries, 0 to 4403
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   item_id      4404 non-null   int64  
 1   name         4404 non-null   object 
 2   description  4401 non-null   object 
 3   rating       4404 non-null   float64
 4   price        4404 non-null   int64  
 5   list_price   4404 non-null   int64  
 6   brand        4404 non-null   object 
 7   group        4404 non-null   object 
 8   url          4404 non-null   object 
 9   image        4404 non-null   object 
dtypes: float64(1), int64(3), object(6)
memory usage: 344.2+ KB


In [4]:
# Number of products
print('The number of products:', products.shape[0])

The number of products: 4404


In [5]:
# Drop duplicate records
print('Trước khi dropping, records =', products.shape[0])
products.drop_duplicates(inplace=True)
print('Sau khi dropping, records =', products.shape[0])

Trước khi dropping, records = 4404
Sau khi dropping, records = 4373


In [6]:
# Check null value
products.isnull().any()

item_id        False
name           False
description     True
rating         False
price          False
list_price     False
brand          False
group          False
url            False
image          False
dtype: bool

In [7]:
# Drop null value
# print('Trước khi dropping, records =', products.shape[0])
# products.dropna(inplace=True)
# print('Sau khi dropping, records =', products.shape[0])

In [7]:
# Save result
products.to_csv('Cung cap HV/Product_new.csv', index=False)

### 2. Review

In [8]:
reviews = pd.read_csv("Cung cap HV/ReviewRaw.csv")
reviews.head()

,customer_id,product_id,name,full_name,created_time,rating,title,content
0,709310,10001012,Lân Nguyễn Hoàng,Lân Nguyễn Hoàng,NaN,3,Ko dùng đc thẻ nhớ,Lúcđầu quên thông tin nên dùng 512gb thì ko đc...
1,10701688,10001012,Nguyễn Khánh Hòa,Nguyễn Khánh Hòa,NaN,5,Cực kì hài lòng,"Tiki giao hàng nhanh. Sản phẩm đúng như mô tả,..."
2,11763074,10001012,Toàn Phạm Khánh,Toàn Phạm Khánh,2019-04-17 15:42:45,5,Cực kì hài lòng,"chất lượng camera rõ nét, chống mưa nắng tuyệt..."
3,9909549,10001012,Nguyen Quang Minh,NaN,NaN,5,Rất hài lòng,"Hàng được đóng gói cẩn thận, giao hàng nhanh ,..."
4,1827148,10001012,Phạm Bá Đức,Phạm Bá Đức,NaN,5,Cực kì hài lòng,"dễ cài đặt, chất lượng tốt, chế độ xem hồng ng..."


In [9]:
# Data type
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364099 entries, 0 to 364098
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   customer_id   364099 non-null  int64 
 1   product_id    364099 non-null  int64 
 2   name          363669 non-null  object
 3   full_name     329471 non-null  object
 4   created_time  117340 non-null  object
 5   rating        364099 non-null  int64 
 6   title         364070 non-null  object
 7   content       165794 non-null  object
dtypes: int64(3), object(5)
memory usage: 22.2+ MB


In [10]:
# Number of reviews
print('Number of reviews:', reviews.shape[0])

Number of reviews: 364099


In [11]:
# Check null value
reviews.isnull().any()

customer_id     False
product_id      False
name             True
full_name        True
created_time     True
rating          False
title            True
content          True
dtype: bool

In [13]:
m = []
for c in ['name','full_name','created_time']:
    m.append({
        'feature':c,
        'MissingVal': reviews[reviews[c].isnull()].shape[0],
        'Percentage': (reviews[reviews[c].isnull()].shape[0]/reviews.shape[0])*100
    })
df = pd.DataFrame(m)
df

,feature,MissingVal,Percentage
0,name,430,0.118100
1,full_name,34628,9.510600
2,created_time,246759,67.772501


Nhận xét:
* created_time có dữ liệu null khá lớn
* full_name có dữ liệu null xấp xỉ 9.5%

In [14]:
# Drop feature created_time and full_name
reviews.drop(['full_name','created_time'],axis=1, inplace=True)

In [15]:
reviews.shape

(364099, 6)

In [16]:
# Save result
reviews.to_csv('Cung cap HV/Review_new.csv', index=False)

### 3. Product - Review

In [17]:
products = pd.read_csv("Cung cap HV/Product_new.csv")
reviews = pd.read_csv("Cung cap HV/Review_new.csv")

In [18]:
# Show products in reviews dataset that is not in products dataset
reviews[~reviews.product_id.isin(products.item_id)]

,customer_id,product_id,name,rating,title,content
156669,5687133,20845056,Lê Phương Tiến,4,Hài lòng,Hài lòng về máy sau 1 tuần dùng. Điểm trừ giao...
156670,8344149,20845056,Luu Tri Dung,5,Tuyệt vời,"Giao hàng đúng thời gian dự kiến, có 2 ngày. M..."
156671,10671105,20845056,Nguyễn Thu Hằng,5,Cực kì hài lòng,"Sản phẩm được đóng gói cẩn thận, pin dùng được..."
156683,373425,20845056,Quân Nguyễn,2,Nên mua ở ngoài,Mình mua kindle Paperwhite 2019 được tầm cỡ 2 ...
156691,1279789,20845056,Quân Trần,1,"Ship chậm, máy ko active được",Tôi nhận đc sản phẩm ko active được. Khi liên ...
...,...,...,...,...,...,...
192248,10449689,12706787,Hai An,4,Hài lòng,có dùng cho dòng điện thoại khác ngoài iphone ...
192249,6079274,12706787,Nguyễn trung hiếu,4,"Hàng nghe mượt, tốt",Hàng chuẩn! Nhưng mình đang phân vân ko biết 1...
192250,7474878,12706787,Mon Pham,5,Quá phê !!!!,Mong lần sau đừng giao hàng chậm nữa tháng như...
192251,13144258,12706787,Đỗ Đức Việt,5,Thắc mắc,Hàng nhập khẩu với hàng VNA khác nhau gì vậy T...


In [19]:
# Drop reviews that have non-existent product_id
reviews = reviews[reviews.product_id.isin(products.item_id)]

In [20]:
reviews.reset_index(drop=True,inplace=True)

In [21]:
reviews.shape[0]

363439

In [22]:
reviews.head()

,customer_id,product_id,name,rating,title,content
0,709310,10001012,Lân Nguyễn Hoàng,3,Ko dùng đc thẻ nhớ,Lúcđầu quên thông tin nên dùng 512gb thì ko đc...
1,10701688,10001012,Nguyễn Khánh Hòa,5,Cực kì hài lòng,"Tiki giao hàng nhanh. Sản phẩm đúng như mô tả,..."
2,11763074,10001012,Toàn Phạm Khánh,5,Cực kì hài lòng,"chất lượng camera rõ nét, chống mưa nắng tuyệt..."
3,9909549,10001012,Nguyen Quang Minh,5,Rất hài lòng,"Hàng được đóng gói cẩn thận, giao hàng nhanh ,..."
4,1827148,10001012,Phạm Bá Đức,5,Cực kì hài lòng,"dễ cài đặt, chất lượng tốt, chế độ xem hồng ng..."


In [23]:
# Save result
reviews.to_csv('Cung cap HV/Review_new.csv',index=False)
products.to_csv('Cung cap HV/Product_new.csv', index=False)